In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, TemplateLibrary

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("my-building")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
TemplateLibrary.load(ontology_graph="../libraries/brick/Brick-subset.ttl")

TemplateLibrary(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7fc0ec023be0>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = TemplateLibrary.load(directory="../libraries/ashrae/guideline36/")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'ztemp', 'zone', 'dat', 'co2', 'dmp', 'occ'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_db985473> ;
    brick:hasPart <urn:my-building/dmp_c07bbdde> ;
    brick:hasPoint <urn:my-building/co2_e8c7fa2a>,
        <urn:my-building/dat_b32277e5>,
        <urn:my-building/occ_894d0931>,
        <urn:my-building/ztemp_aeea93ff> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_5bc1ae1c> ;
    brick:hasPart <urn:my-building/dmp_1f1aea36> ;
    brick:hasPoint <urn:my-building/co2_0e147480>,
        <urn:my-building/dat_9e1a0a03>,
        <urn:my-building/occ_dfce93ae>,
        <urn:my-building/ztemp_f99c9e18> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_92de96ef> ;
    brick:hasPart <urn:my-building/dmp_9cd4db7c> ;
    brick:hasPoint <urn:my-building/co2_7184c806>,
        <urn:my-building/dat_e3e73b4f>,
        <urn:my-building/occ_8e0cb078>,
        <urn:my-building/ztemp_6823b81b> .



In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_d5db4fab> a brick:VAV ;
    brick:feeds <urn:my-building/zone_a06bae25> ;
    brick:hasPart <urn:my-building/dmp_5dfb24b6> ;
    brick:hasPoint <urn:my-building/co2_f1847852>,
        <urn:my-building/dat_f9b84eab>,
        <urn:my-building/occ_add6f2a6>,
        <urn:my-building/ztemp_e0e75e14> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_fcbf0f1b> a brick:VAV ;
    brick:feeds <urn:my-building/zone_5120076d> ;
    brick:hasPart <urn:my-building/dmp_ff36c332> ;
    brick:hasPoint <urn:my-building/co2_6c6f9535>,
        <urn:my-building/dat_a374adec>,
        <urn:my-building/occ_67ff6b45>,
        <urn:my-building/ztemp_61eb02ff> .

<urn:my-building/co2_6c6f9535> a brick:CO2_Level_Sensor .

<urn:my-building/dat_a374adec> a brick:Discharge_Air_Temperature_Sensor .

<urn:my-building/dmp_ff36c332> a brick:Damper ;
    brick:hasPoint <urn:my-building/dmppos-damperb6e126bc-inlined_85aebf55> .

<urn:my-building/dmppos-damperb6e126bc-inlined_85aebf55> a brick:Damper_Position_Command .

<urn:my-building/occ_67ff6b45> a brick:Occupancy_Sensor .

<urn:my-building/zone_5120076d> a brick:HVAC_Zone .

<urn:my-building/ztemp_61eb02ff> a brick:Zone_Air_Temperature_Sensor .




In [14]:
print(inlined.body.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:VAV ;
    brick:feeds <urn:___param___#zone> ;
    brick:hasPart <urn:___param___#dmp> ;
    brick:hasPoint <urn:___param___#co2>,
        <urn:___param___#dat>,
        <urn:___param___#occ>,
        <urn:___param___#ztemp> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos-damperb6e126bc-inlined> .

<urn:___param___#dmppos-damperb6e126bc-inlined> a brick:Damper_Position_Command .

<urn:___param___#occ> a brick:Occupancy_Sensor .

<urn:___param___#zone> a brick:HVAC_Zone .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .




In [15]:
lib = TemplateLibrary.load(directory="../libraries/bca-chiller-plant/")

In [16]:
chiller = lib.get_template_by_name("chiller")
inlined = chiller.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<urn:my-building/name_51ab6c17> a brick:Chiller ;
    brick:hasPart <urn:my-building/chwp_90faa89c>,
        <urn:my-building/cnd_43ee08d7>,
        <urn:my-building/cvlv_86942d1b> ;
    brick:hasPoint <urn:my-building/chwrt_e2d2467a>,
        <urn:my-building/chwst_a2a24d3e> ;
    brick:isMeteredBy <urn:my-building/meter_2499e921> .

<urn:my-building/breaker_status-electrical_meter458a2b36-inlined_53c85c6f> a <breaker_status> .

<urn:my-building/breaker_status-electrical_meter76abaa35-inlined_916e2d47> a <breaker_status> .

<urn:my-building/breaker_status-electrical_meterb8ef2746-inlined_4da7f304> a <breaker_status> .

<urn:my-building/bypass_valve-pump2b7a7593-inlined_062f9530> a brick:Bypass_Valve ;
    brick:hasPoint <urn:my-building/vlv_cmd-bypass_valvef0a93f36-inlined_ca091902> .

<urn:my-building/bypass_valve-pumpc884ebbc-inlined_f388e5ab> a brick:Bypass_Valve ;
    